# Exercise 1: PCA, ICA and Sparse Coding

**Note**: Please insert the names of all participating students:

1. 
2. 
3. 
4. 
5. 

## Preamble
The following code downloads and imports all necessary files and modules into the virtual machine of Colab. Please make sure to execute it before solving this exercise. This mandatory preamble will be found on all exercise sheets.

In [ ]:
import sys, os
if 'google.colab' in sys.modules:
  if os.getcwd() == '/content':
    !git clone 'https://github.com/inb-uni-luebeck/cs5450.git'
    os.chdir('cs5450')

## Exercise 1.1: Applying unsupervised methods

In this exercise, we will apply three unsupervised methods, namely *Prinicipal Component Analysis (PCA)*, *Independent Component Analysis (ICA)*, and *Sparse Coding (SC)* to image patches taken from handwritting digits and from faces.

Since implementing [ICA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html) and [SC](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparseCoder.html) is rather complex, we will use already existing python packages.

We will build on the python starter code given below. You only need to write code at the places indicated by `TODO`.
The two given datasets are `mnist.npz` for handwritten digits and `olivetti.npz` for faces. Each dataset contains a multi dimensional array `data` with the first index corresponding to the observations and the second index corresponding to the features (grayscale pixels). Both datasets contain square-shaped images.

Implement the missing parts below. When done correctly, your code should do the following:
1. Load the dataset with either handwritten images or faces.
2. Enforce mean-free observations by subtracting the mean from the data.
3. Randomly draw square-shaped image patches from the data.
4. Apply one of the three unsupervised methods (PCA, ICA, or SC).
5. Show the resulting set of new basis vectors (i.e. the components for PCA and ICA).
6. Reconstruct the original image patches with different subsets of basis vectors.

In case you are struggeling with the task, here are some helpful tips and hints:
1. Useful functions: [`numpy.load`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.load.html), [`ndarray.astype`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.astype.html), [`numpy.mean`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html), [`numpy.linalg.pinv`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.pinv.html), [`numpy.matmul`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html), [`numpy.stack`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.stack.html), `random_patches`, `show_in_grid`, [`PCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html), [`FastICA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html#sklearn.decomposition.FastICA), [`DictionaryLearning`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.DictionaryLearning.html#sklearn.decomposition.DictionaryLearning)
2. You should start with both PCA and ICA and only then use SC.
3. Applying SC may take a couple of minutes. Good initial value for it are: 
 - `dataset='mnist'`
 - `n_patches=10000`
 - `patch_size=25`
 - `n_basis=100`
 - `n_iterations=50`
 - and instantiate the class with `n_jobs=-1` to utilize all CPU cores, as well as `verbose=True` to see progress feedback

Import and helper functions. Do not change!

In [1]:
import numpy as np
import math
import random
from sklearn.decomposition import PCA, FastICA, DictionaryLearning
from matplotlib import pyplot as plt
%matplotlib inline

def random_patches(data, img_height, img_width, n_patches, patch_size):
    # RANDOMPATCHES draw random square-shaped patches from the data.
    #
    # INPUT:
    #   data : ndarray containing the data (i.e. mnist or olivetti)
    #   img_height : height of the images
    #   img_width : width of the images
    #   n_patches : number of patches to be drawn
    #   patch_size : size (both width and height) of the square-shaped patches
    #
    # OUTPUT:
    #   patches : random patches

    # reshape data 2d -> 3d
    n_images = data.shape[0]
    data = data.reshape(n_images, img_height, img_width)

    # extract random patches
    patches = []#np.zeros((n_patches, patch_size*patch_size))
    for i in range(n_patches):
        patch_top = random.randint(0, img_height - patch_size)
        patch_left = random.randint(0, img_width - patch_size)
        p = data[random.randint(0, n_images-1), patch_top:patch_top+patch_size, patch_left:patch_left+patch_size]
        patches.append(p)
        #patches[i,:] = p[:]
        
    patches = np.stack(patches)
    patches = patches.reshape(patches.shape[0], -1) #flatten each patch
        
    return patches

def show_in_grid(images, height, width):
    # flatten images if necessary
    images = images.reshape(images.shape[0], -1)
    
    # normalize patches
    images = images / (np.abs(images).max(axis=1, keepdims=True)+1e-6)
    
    # reshape into images
    images = images.reshape(-1, height, width)
    
    # make images fit into a rectangular area
    grid_width = math.ceil(math.sqrt(images.shape[0]))
    grid_height = math.ceil(images.shape[0] / grid_width)
    
    empty_cells = grid_width * grid_height - images.shape[0]
    
    # fill empty cells
    if empty_cells > 0:
        padding = np.zeros((empty_cells, height, width))
        images = np.concatenate((images, padding))
        
    # rearrange basis into grid and also switch width and height so x and y axis are not switched
    images = images.reshape(grid_height, grid_width, height, width)
    images = images.transpose(0, 3, 1, 2)
    
    plt.figure(figsize = (10,10))
    plt.imshow(images.reshape(grid_height*height, grid_width*width), cmap='gray')
    plt.show()

Your solution:

In [2]:
## Define variables

# TODO dataset : relative path to the .npz file containing the data
dataset = '' #'data/olivetti.npz'

# TODO n_patches : number of random patches to extract from the images
n_patches = -1
# TODO patch_size : size (both width and height) of the square-shaped patches
patch_size = -1

# TODO n_basis : number of new basis vectors
n_basis = -1

# TODO method : string describing whether to perform pca, ica, or sc
method = ''

# TODO n_iterations : number of iterations (only for sparse coding)
n_iterations = -1

In [3]:
## Compute and show the principal components

# TODO load dataset

# TODO convert data to floating point (double precision)

# TODO get number of images (observations) and number of pixels (features)
n_images, n_pixels = -1,-1

# TODO get height and width of the images
img_height = -1
img_width = -1
assert isinstance(img_height, int) and isinstance(img_width, int), "Image dimensions need to be integers."

# TODO enforce mean-free data vectors
mean_free_data = -1

# TODO draw random patches from data.
patches = -1

# TODO perform the chosen unsupervised method on the drawn patches
if method == 'pca':
    basis = -1
elif method == 'ica':
    basis = -1
elif method == 'sc':
    basis = -1

# TODO show the new set of basis vectors (only the first n_basis)
#show_in_grid(basis[:n_basis], patch_size, patch_size)

In [4]:
## Use different subsets of the new basis vectors to reconstruct a random patch
from sklearn.decomposition import sparse_encode

# TODO draw a random patch
random_patch = -1

# TODO reconstruct the random patch
reconstructions = []
for i in range(n_basis):
    #sub_basis = basis[0:i+1]
    
    if method == 'sc':
        pass
    else:
        pass
    
#reconstructions = np.stack(reconstructions)

# TODO show the reconstructed patch
#show_in_grid(reconstructions, patch_size, patch_size)

*Bonus*: Implement the missing parts in the bonus section. When done correctly, your code additionally should do the following:
1. Perform ZCA whitening, whereas $\vec{x}_{ZCAwhite}=U\vec{x}_{wht}$ (c.f. equation 1.3 in the lecture notes).
2. Visualize the whitened data.

In [5]:
#% TODO epsilon : avoid negative numbers while computing stdDeviations
epsilon = 0.1

# TODO perform principal component analysis (help pca)
basis = -1 #get basis vector components

# TODO compute the covariance matrix of the patches
covariance = -1

# TODO compute diagonal matrix S containing the standard deviations (c.f. page 6)
std_deviations = -1

# TODO pca whitening (c.f. equation 1.3)
xPCAwhite = -1

# TODO zca whitening
xZCAwhite = -1

# TODO visualize the whitened data
# show_in_grid(xZCAwhite[:n_basis], patch_size, patch_size)

## Exercise 1.2: Comprehension Questions

Answer the following comprehension questions either with *right* or *wrong* and briefly explain your decision:
1. Regardless of the used method for representation learning (i.e. PCA, ICA, SC), the resulting set of basis vectors depends on the given dataset.

[Your answer]

2. The first principal component has the smallest eigenvalue and covers the largest possible variance of the data.

[Your answer] 

3. The optimal number of principal components to retain is 42.

 [Your answer]

4. After whitening, the features of an observation are uncorrelated and have unit variance.

[Your answer] 

5. The resulting basis vectors of the ICA (i.e. the independent components) are mutually orthogonal.

 [Your answer]

6. The ICA is an enhancement of the PCA which additionally makes the already uncorrelated features statistically independent.

 [Your answer]

7. The set of basis vectors of the ICA can be over-complete, i.e. more basis vectors than dimensions of the input space.

 [Your answer]

8. Sparsity means that only a few basis vectors are used to reconstruct an image.

 [Your answer]

9. The optimization problem for SC (c.f. equation 1.9 in the lecture notes) is convex.

[Your answer] 

10. Unlike PCA and ICA, SC results in a linear transform of the data.

[Your answer]